In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
import os

In [2]:
path_t = ""
dir_t_list = os.listdir(path_t)

path_r = ""
dir_r_list = os.listdir(path_r)

In [3]:
temp = []

for i in range(1980,2016,5):
    current_10_temp = []
    for j in range(0,10):
        current_temp = pd.read_csv(path_t+'/OBS_ASOS_TIM_' + str(i+j) + '.csv', encoding='cp949')
        current_10_temp.append(current_temp)
    temp.append(current_10_temp)

    
rain = []

for i in range(1980,2016,5):
    current_10_rain = []
    for j in range(0,10):
        current_rain = pd.read_csv(path_r+'/OBS_ASOS_TIM_rain_' + str(i+j) + '.csv', encoding='cp949')
        current_10_rain.append(current_rain)
    rain.append(current_10_rain)
    

print(len(temp))
print(len(rain))

8
8


In [4]:
siteset = ['속초','대관령','강릉','울진','포항','영덕','춘천','서울','인천','원주','수원','충주','서산','청주','대전','양평','이천','인제','홍천','제천','보은','보령','부여','금산','추풍령','군산','대구','전주','울산','광주','부산','통영','목포','여수','완도','제주','성산','서귀포','진주','강화','천안','부안','임실','정읍','남원','해남','고흥','영주','문경','의성','구미','영천','거창','합천','밀양','산청','거제','남해']

In [5]:
def KoppenClassification_V(tempData, rainData):
    values = {}
    
    #monthly average precipitation
    rainData.columns = rainData.columns.str.strip()
    rainData['일시'] = pd.to_datetime(rainData['일시'])
    rainData['강수량(mm)'] = rainData['강수량(mm)'].fillna(0)

    rainData['연월'] = rainData['일시'].dt.to_period('M')

    daily_rain = rainData.groupby(rainData['일시'].dt.date)['강수량(mm)'].sum().reset_index()
    daily_rain['연월'] = pd.to_datetime(daily_rain['일시']).dt.to_period('M')

    monthly_total = daily_rain.groupby('연월')['강수량(mm)'].sum()   #monthly total

    days_per_month = daily_rain.groupby('연월')['일시'].nunique()    #days per month

    monthly_avg_rain_list = (monthly_total / days_per_month).tolist()     #average rain list
    
    values['monthly_avg_rain_list'] = monthly_avg_rain_list
    
    #월평균기온(리스트)
    tempData.columns = tempData.columns.str.strip()
    tempData['일시'] = pd.to_datetime(tempData['일시'])

    tempData['일자'] = tempData['일시'].dt.date

    daily_avg_temp = tempData.groupby('일자')['기온(°C)'].mean().reset_index()
    
    daily_avg_temp['연월'] = pd.to_datetime(daily_avg_temp['일자']).dt.to_period('M')

    monthly_avg_temp_from_daily = daily_avg_temp.groupby('연월')['기온(°C)'].mean()

    monthly_avg_temp_list = monthly_avg_temp_from_daily.tolist()
    
    annual_av_temp = sum(monthly_avg_temp_list)/len(monthly_avg_temp_list)
    
    values['annual_av_temp'] = annual_av_temp
    
    values['monthly_avg_temp_list'] = monthly_avg_temp_list
    
    #최한월기온
    coldest_av_temp = min(monthly_avg_temp_list)
    
    values['coldest_av_temp'] = coldest_av_temp

    #최난월기온
    warmest_av_temp = max(monthly_avg_temp_list)
    
    values['warmest_av_temp'] = warmest_av_temp

    #연강수량
    rainData.columns = rainData.columns.str.strip()
    rainData['일시'] = pd.to_datetime(rainData['일시'])
    rainData['일자'] = rainData['일시'].dt.date
    daily_rain = rainData.groupby('일자')['강수량(mm)'].sum()

    annual_precipitation_amount = daily_rain.sum()
    
    values['annual_precipitation_amount'] = annual_precipitation_amount

    #threshold1
    rainData['일시'] = pd.to_datetime(rainData['일시'])
    rainData['월'] = rainData['일시'].dt.month
    
    annual_avg_temp = tempData['기온(°C)'].mean()
    
    april_to_september_precip = rainData[rainData['월'].between(4, 9)]['강수량(mm)'].sum()
    
    summer_ratio = april_to_september_precip / annual_precipitation_amount
    
    if summer_ratio >= 0.7:
        add = 140
    elif summer_ratio >= 0.3:
        add = 70
    else:
        add = 0
        
    threshold1 = annual_avg_temp*10+add
    
    values['threshold1'] = threshold1
    
    #threshold2
    rainData['일시'] = pd.to_datetime(rainData['일시'])
    rainData['월'] = rainData['일시'].dt.month
    
    annual_avg_temp = tempData['기온(°C)'].mean()
    
    april_to_september_precip = rainData[rainData['월'].between(4, 9)]['강수량(mm)'].sum()
        
    values['april_to_september_precip'] = april_to_september_precip
    
    if summer_ratio >= 0.7:
        add2 = 280
    elif summer_ratio >= 0.3:
        add2 = 140
    else:
        add2 = 0
        
    threshold2 = annual_avg_temp*20+add2
    
    values['threshold2'] = threshold2

    wettest_month_list = monthly_avg_rain_list[0:2]
    wettest_month_list.append(monthly_avg_rain_list[-1])
    wettest_month_122 = max(wettest_month_list)
    
    values['wettest_month_122'] = wettest_month_122
    
    driest_month_68 = min(monthly_avg_rain_list[5:8])
    
    values['driest_month_68'] = driest_month_68
    
    driest_month_list = monthly_avg_rain_list[0:2]
    driest_month_list.append(monthly_avg_rain_list[-1])
    driest_month_122 = min(driest_month_list)
    
    values['driest_month_122'] = driest_month_122

    wettest_month_68 = max(monthly_avg_rain_list[5:8])
    
    values['wettest_month_68'] = wettest_month_68

    count_10 = sum(1 for t in monthly_avg_temp_list if t > 10)

    values['count_10'] = count_10
    
    return values

In [6]:
def tenYearVariables(valuesList):
    tenYearVariable = {}

    #10년 최한월 평균
    coldestTen = []
    for i in range(0,10):
        coldestTen.append(valuesList[i]['coldest_av_temp'])
    coldest_av_temp_10 = sum(coldestTen)/len(coldestTen)

    tenYearVariable['coldest_av_temp_10'] = coldest_av_temp_10

    #10년 최난월 평균
    warmestTen = []
    for i in range(0,10):
        warmestTen.append(valuesList[i]['warmest_av_temp'])
    warmest_av_temp_10 = sum(warmestTen)/len(warmestTen)

    tenYearVariable['warmest_av_temp_10'] = warmest_av_temp_10

    #10년 연평균 강수량
    annualRainTen = []
    for i in range(0,10):
        annualRainTen.append(valuesList[i]['annual_precipitation_amount'])
    annual_precipitation_amount_10 = sum(annualRainTen)/len(annualRainTen)

    tenYearVariable['annual_precipitation_amount_10'] = annual_precipitation_amount_10

    #10년 연평균 기온
    annualTempTen = []
    for i in range(0,10):
        annualTempTen.append(valuesList[i]['annual_av_temp'])
    annual_av_temp_10 = sum(annualTempTen)/len(annualTempTen)

    tenYearVariable['annual_av_temp_10'] = annual_av_temp_10

    #10년 AprilSeptember
    annualAprSep = []
    for i in range(0,10):
        annualAprSep.append(valuesList[i]['april_to_september_precip'])
    annualAprSep_10 = sum(annualAprSep)/len(annualAprSep)

    tenYearVariable['annualAprSep_10'] = annualAprSep_10

    #threshold1_10
    summer_ratio = annualAprSep_10 / annual_precipitation_amount_10
    if summer_ratio >= 0.7:
        add = 140
    elif summer_ratio >= 0.3:
        add = 70
    else:
        add = 0

    threshold1_10 = annual_av_temp_10*10+add
    tenYearVariable['threshold1_10'] = threshold1_10


    #threshold2_10
    if summer_ratio >= 0.7:
        add2 = 280
    elif summer_ratio >= 0.3:
        add2 = 140
    else:
        add2 = 0
    threshold2_10 = annual_av_temp_10*20+add2
    tenYearVariable['threshold2_10'] = threshold2_10

    #wettest_month_122_10
    wettest_month_122_10_List = []
    for i in range(0,10):
        wettest_month_122_10_List.append(valuesList[i]['wettest_month_122'])
    wettest_month_122_10 = sum(wettest_month_122_10_List)/len(wettest_month_122_10_List)

    tenYearVariable['wettest_month_122_10'] = wettest_month_122_10

    #driest_month_122_10
    driest_month_122_10_List = []
    for i in range(0,10):
        driest_month_122_10_List.append(valuesList[i]['driest_month_122'])
    driest_month_122_10 = sum(driest_month_122_10_List)/len(driest_month_122_10_List)

    tenYearVariable['driest_month_122_10'] = driest_month_122_10

    #wettest_month_68_10
    wettest_month_68_10_List = []
    for i in range(0,10):
        wettest_month_68_10_List.append(valuesList[i]['wettest_month_68'])
    wettest_month_68_10 = sum(wettest_month_68_10_List)/len(wettest_month_68_10_List)

    tenYearVariable['wettest_month_68_10'] = wettest_month_68_10

    #driest_month_68_10
    driest_month_68_10_List = []
    for i in range(0,10):
        driest_month_68_10_List.append(valuesList[i]['driest_month_68'])
    driest_month_68_10 = sum(driest_month_68_10_List)/len(driest_month_68_10_List)

    tenYearVariable['driest_month_68_10'] = driest_month_68_10

    #count_10_10
    count_10_lists = []
    for i in range(0,10):
        count_10_lists.append(valuesList[i]['monthly_avg_temp_list'])
    monthly_avg_temp_list_10 = [sum(values) / 10 for values in zip(*count_10_lists)]
    count_10_10 = sum(1 for t in monthly_avg_temp_list_10 if t > 10)
    tenYearVariable['count_10_10'] = count_10_10

    
    return tenYearVariable

In [7]:
def KoppenClassification_10y_variable(result):
    koppen_list = []

    for tenYearVariable in result:
        coldest_av_temp = tenYearVariable['coldest_av_temp_10']
        warmest_av_temp = tenYearVariable['warmest_av_temp_10']
        annual_precipitation_amount = tenYearVariable['annual_precipitation_amount_10']
        threshold1 = tenYearVariable['threshold1_10']
        threshold2 = tenYearVariable['threshold2_10']
        wettest_month_122 = tenYearVariable['wettest_month_122_10']
        driest_month_122 = tenYearVariable['driest_month_122_10']
        wettest_month_68 = tenYearVariable['wettest_month_68_10']
        driest_month_68 = tenYearVariable['driest_month_68_10']
        count_10 = tenYearVariable['count_10_10']

        # A
        if coldest_av_temp > 18:
            if annual_precipitation_amount > threshold1:
                koppen_list.append("A")
                continue

        # B
        if threshold1<annual_precipitation_amount < threshold2:
            koppen_list.append("BW")
            continue
        elif annual_precipitation_amount < threshold1:
            koppen_list.append("BS")
            continue

        # C
        if -3 < coldest_av_temp < 18:
            if annual_precipitation_amount > threshold1:
                if wettest_month_122 > 3 * driest_month_68:
                    koppen_list.append("Cs")
                    continue
                elif wettest_month_68 > driest_month_122 * 10:
                    koppen_list.append("Cw")
                    continue
                elif warmest_av_temp > 22:
                    koppen_list.append("Cfa")
                    continue
                elif warmest_av_temp < 22:
                    if count_10 >= 4:
                        koppen_list.append("Cfb")
                        continue
                    elif 1 <= count_10 <= 3 and coldest_av_temp > -38:
                        koppen_list.append("Cfc")
                        continue

        # D
        if warmest_av_temp > 10 and coldest_av_temp <= -3:
            if annual_precipitation_amount > threshold1:
                if wettest_month_68 > 10 * driest_month_122:
                    koppen_list.append("Dw")
                    continue
                elif warmest_av_temp > 22:
                    koppen_list.append("Dfa")
                    continue
                elif warmest_av_temp < 22:
                    if count_10 >= 4:
                        koppen_list.append("Dfb")
                        continue
                    elif 1 <= count_10 <= 3 and coldest_av_temp > -38:
                        koppen_list.append("Dfc")
                        continue

        # E
        if warmest_av_temp < 10:
            koppen_list.append("E")
            continue

        # 기타 분류 실패한 경우
        koppen_list.append("Unclassified")

    return koppen_list


In [8]:
def Koppen(region):
    region_temp_list = []
    for group in temp:
        region_temp_list.append([df[df['지점명'] == region].copy() for df in group])

    region_rain_list = []
    for group in rain:
        region_rain_list.append([df[df['지점명'] == region].copy() for df in group])

    valuesList = []
    for temp_group, rain_group in zip(region_temp_list, region_rain_list):
        values = [KoppenClassification_V(t, r) for t, r in zip(temp_group, rain_group)]
        valuesList.append(values)

    ten_year_results = [tenYearVariables(v) for v in valuesList]
    return KoppenClassification_10y_variable(ten_year_results)

In [10]:
for i in siteset:
    print(i,Koppen(i))

속초 ['Cw', 'Cfa', 'Cfa', 'Cw', 'Cw', 'Cw', 'Cw', 'Cw']
대관령 ['Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw']
강릉 ['Cw', 'Cfa', 'Cfa', 'Cfa', 'Cw', 'Cw', 'Cfa', 'Cw']
울진 ['Cfa', 'Cfa', 'Cfa', 'Cfa', 'Cw', 'Cfa', 'Cfa', 'Cfa']
포항 ['Cfa', 'Cfa', 'Cfa', 'Cw', 'Cw', 'Cfa', 'Cfa', 'Cw']
영덕 ['Cw', 'Cfa', 'Cw', 'Cw', 'Cw', 'Cfa', 'Cfa', 'Cw']
춘천 ['Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw']
서울 ['Dw', 'Cw', 'Cw', 'Cw', 'Cw', 'Cw', 'Dw', 'Cw']
인천 ['Dw', 'Cw', 'Cw', 'Cw', 'Cw', 'Cw', 'Cw', 'Cw']
원주 ['Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Cw']
수원 ['Dw', 'Cw', 'Cw', 'Cw', 'Cw', 'Cw', 'Dw', 'Cw']
충주 ['Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw']
서산 ['Cw', 'Cw', 'Cw', 'Cw', 'Cw', 'Cw', 'Cw', 'Cw']
청주 ['Dw', 'Cw', 'Cw', 'Cw', 'Cw', 'Cw', 'Cw', 'Cw']
대전 ['Cw', 'Cw', 'Cw', 'Cw', 'Cw', 'Cw', 'Cw', 'Cw']
양평 ['Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Cw']
이천 ['Dw', 'Dw', 'Cw', 'Dw', 'Dw', 'Dw', 'Dw', 'Cw']
인제 ['Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw']
홍천 ['Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'Dw', 'D